In [1]:
import pandas as pd
from pprint import pprint
import json

df = pd.read_json("data/train.jsonl", lines=True)

training_data = []
for row in df.itertuples():
    input = json.loads(row.instruction)
    output = json.loads(row.output)

    output["content"] = output["function_call"]["arguments"]
    del output["function_call"]

    sample = {"messages": input.copy() + [output]}
    training_data.append(sample)

# save the training data to data/train-gpt3.5.jsonl

with open("data/train-gpt3.5.jsonl", "w") as f:
    for sample in training_data:
        f.write(json.dumps(sample) + "\n")

print(f"Sample training data:")
pprint(training_data[0])


Sample training data:
{'messages': [{'content': 'Your goal is to classify a recipe along several '
                          'dimensions.Pay attention to the instructions.',
               'role': 'system'},
              {'content': 'Homemade Salad Dressing\n'
                          '\n'
                          'Ingredients:\n'
                          "- 1 pt. Hellmann's mayonnaise\n"
                          '- 1 pt. buttermilk\n'
                          '- 1 tsp. Accent\n'
                          '- 2 Tbsp. dry parsley\n'
                          '- 2 pkg. low-calorie Italian salad dressing mix\n'
                          '- 1 can jalapeno peppers or 4 oz. Jimenez green '
                          'sauce\n'
                          '\n'
                          'Directions:\n'
                          '- Blend well in blender; store in refrigerator.\n'
                          '- For dip, decrease liquid.',
               'role': 'user'},
              {'content': 

In [28]:
import os
import openai

import dotenv

dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

openai.File.create(
    file=open("data/train-gpt3.5.jsonl", "rb"),
    purpose="fine-tune",
    user_provided_filename="recipe-classification",
)


<File file id=file-faAdQ1KPxZH79ThW4Dbu4z1y at 0x7fa55db5c6d0> JSON: {
  "object": "file",
  "id": "file-faAdQ1KPxZH79ThW4Dbu4z1y",
  "purpose": "fine-tune",
  "filename": "recipe-classification",
  "bytes": 4210831,
  "created_at": 1693000959,
  "status": "uploaded",
  "status_details": null
}

In [40]:
openai.File.list()


<OpenAIObject list at 0x7fa55dbf6930> JSON: {
  "object": "list",
  "data": [
    {
      "object": "file",
      "id": "file-faAdQ1KPxZH79ThW4Dbu4z1y",
      "purpose": "fine-tune",
      "filename": "recipe-classification",
      "bytes": 4210831,
      "created_at": 1693000959,
      "status": "processed",
      "status_details": null
    }
  ]
}

In [42]:
openai.FineTuningJob.create(
    training_file="file-faAdQ1KPxZH79ThW4Dbu4z1y", model="gpt-3.5-turbo"
)


<FineTuningJob fine_tuning.job id=ftjob-EjjLxmj9P8apwPRk5s2NPSeB at 0x7fa55ddc4360> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-EjjLxmj9P8apwPRk5s2NPSeB",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693001190,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-jRz4nVPMoeGHWL5nVR3Mb0kp",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-faAdQ1KPxZH79ThW4Dbu4z1y",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}